<a href="https://www.kaggle.com/code/kurshidbasheer/eda-stanford-rna3d-folding-2?scriptVersionId=298213420" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os

print("Top level folders:")
print(os.listdir('/kaggle/input'))


In [ ]:
os.listdir('/kaggle/input/stanford-rna-3d-folding-2')


In [ ]:
train_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv')
train_labels.head()

In [ ]:
train_labels['copy'].unique()

In [ ]:
train_labels.groupby('ID').size().head()

In [ ]:
train_labels.groupby('copy').size()

In [ ]:
#imp
train_labels.groupby(['ID','copy']).size().head()

Pandas detected mixed datatype in column 6

In [ ]:
train_labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

No of copies for a single sequence

In [ ]:
train_labels['seq_id'] = train_labels['ID'].str.split('_').str[0]

train_labels.groupby('seq_id')['copy'].nunique().describe()

train_labels.groupby('seq_id')['copy'].max().describe()

Analyze Sequence Length Distribution

In [ ]:
train_seq = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv')

train_seq['length'] = train_seq['sequence'].str.len()

train_seq['length'].describe()


In [ ]:
train_seq['length'].hist(bins=50)


How many sequences are truly massive?

In [ ]:
(train_seq['length'] > 5000).sum()


In [ ]:
(train_seq['length'] > 10000).sum()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
    len(os.listdir('/kaggle/input/stanford-rna-3d-folding-2/PDB_RNA'))


In [ ]:
train = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv')
test = pd.read_csv('/kaggle/input/stanford-rna-3d-folding-2/test_sequences.csv')

print("Train max length:", train['sequence'].apply(len).max())
print("Test max length:", test['sequence'].apply(len).max())


The error tells us something important:

'sequence_id' is not a column name in train_labels.csv.

In [ ]:
labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

print(labels.columns)


In [ ]:
labels.head()

inspect actual column names in train_labels.csv

How many sequences in train are >5000 nt?

In [ ]:
(train['sequence'].apply(len) > 5000).sum()


Compute Labeled Residues Per Sequence

In [ ]:
labels = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_labels.csv',
    low_memory=False
)

labels_per_seq = labels.groupby('ID').size()

print("Max labeled residues:", labels_per_seq.max())
print("Median labeled residues:", labels_per_seq.median())
print("Min labeled residues:", labels_per_seq.min())
print("Number of sequences:", labels_per_seq.shape[0])

how many conformations per RNA?

In [ ]:
labels['copy'].value_counts()

In [ ]:
labels.head(10)


In [ ]:
labels['ID'].head(10)


In [ ]:
labels['seq_id'] = labels['ID'].apply(lambda x: x.split('_')[0])


In [ ]:
labels_per_seq = labels.groupby('seq_id').size()

print("Max labeled residues:", labels_per_seq.max())
print("Median labeled residues:", labels_per_seq.median())
print("Min labeled residues:", labels_per_seq.min())
print("Number of sequences:", labels_per_seq.shape[0])


In [ ]:
(train_seq['length'] < 300).sum()
(train_seq['length'] < 1000).sum()


In [ ]:
import pandas as pd

train_seq = pd.read_csv(
    '/kaggle/input/stanford-rna-3d-folding-2/train_sequences.csv'
)

train_seq['length'] = train_seq['sequence'].apply(len)


In [ ]:
print("Total sequences:", len(train_seq))

print("< 300 nt:", (train_seq['length'] < 300).sum())
print("< 1000 nt:", (train_seq['length'] < 1000).sum())
print("< 2000 nt:", (train_seq['length'] < 2000).sum())
print("> 5000 nt:", (train_seq['length'] > 5000).sum())


There are extreme outliers (125k nt)

Those are rare (very few sequences).
Likely ribosomal RNA.

You must confirm how many such extreme cases exist:

check if P or c4 exists (forchirality constraints)

In [ ]:
type(train_seq)
train_seq.columns
train_seq.head(1)

In [ ]:
print(train_labels.columns)


In [ ]:
import os

# IDs from train_sequences.csv
seq_ids = set(train_seq["target_id"].str.upper())

# IDs from rna pdb folder
cif_ids = set([f.split(".")[0].upper()
               for f in os.listdir("/kaggle/input/stanford-rna-3d-folding-2/PDB_RNA")
               if f.endswith(".cif")])

print("Total sequence IDs:", len(seq_ids))
print("Total CIF IDs:", len(cif_ids))

print("Overlap:", len(seq_ids & cif_ids))
print("Missing in CIF:", len(seq_ids - cif_ids))
print("Extra in CIF:", len(cif_ids - seq_ids))


Compute GT (ground Truth ) labels to store sign per residue

Minimal mmCIF Parser